#### Cryptography
#### Task № 3 - G(H)OST 28147-89
#### Ivan Rybin - ITMO JB SE MA 2021

In [1]:
import gost_data

In [2]:
def xor_bytes(l, r):
    return bytearray([lb ^ rb for lb, rb in zip(l, r)])

def xor_bits(l, r):
    return "".join([str(int(lb) ^ int(rb)) for lb, rb in zip(l, r)])

def stobytes(s):
    return s.encode('utf-8')

def sto64(bits):
    while len(bits) % 64 != 0:
        bits += '0'
    return bits

def to8bits(bits):
    while len(bits) < 8:
        bits = '0' + bits
    return bits

def to4bits(bits):
    while len(bits) < 4:
        bits = '0' + bits
    return bits

def stobits(s):
    b = ""
    for c in stobytes(s):
        b += to8bits(bin(c)[2:])
    return b

def btoi(b):
    i = 0
    k = 0
    for c in reversed(b):
        i += int(c) * (2 ** k)
        k += 1
    return i

def bytestos(b):
    return b.decode('utf-8')

def bitstos(bits):
    s = []
    for b in split_to_blocks(bits, 8):
        s.append(btoi(b))
    return bytearray(s)

def perm(p, b):
    return ''.join([b[i] for i in p])

def split_to_blocks(b, block_size):
    blocks = []
    for i in range(1, int((len(b) + block_size) / block_size)):
        blocks.append(b[(i - 1) * block_size: i * block_size])
    return blocks


def cyclic_offset(b, offset, is_left=True):
    new_b = ''
    if is_left:
        for i in range(offset, offset + len(b)):
            new_b += b[i % len(b)]
    else:
        for i in range(0, offset):
            b = b[-1] + b[:-1]
        new_b = b
    return new_b

In [3]:
def get_8sub_keys(gost_key):
    return [gost_key[i * 32: (i + 1) * 32] for i in range(0, 8)]


def sbox_val(sid, bits):
    col = btoi(bits)
    return gost_data.s_boxes[(sid * 16) + col]


def sboxes_perm(data32):
    new_data = ''
    for sbox_id in range(0, 8):
        new_block = data32[sbox_id * 4: (sbox_id + 1) * 4]
        new_block = sbox_val(sbox_id, new_block)
        new_block = to4bits(bin(new_block)[2:])
        new_data += new_block
    return new_data


def enc_func(data32, key32):
    assert (len(data32) == 32 and len(key32) == 32)
    data32 = xor_bits(data32, key32)
    data32 = sboxes_perm(data32)
    data32 = cyclic_offset(data32, 11)
    return data32

In [4]:
def GOST_round(block, key):
    n2, n1 = block[0:32], block[32:]
    s = enc_func(n1, key)
    s = xor_bits(n2, s)
    return n1, s


def GOST_ECB(key, message, encrypt=True):
    blocks = split_to_blocks(message, 64)
    keys = None
    sub_keys = get_8sub_keys(key)
    if encrypt:
        keys = sub_keys * 3 + list(reversed(sub_keys))
    else:
        keys = sub_keys + list(reversed(sub_keys)) * 3
    
    new_blocks = []
    for i in range(0, len(blocks)):
        block = blocks[i]
        for i in range(0, 32):
            n2, n1 = GOST_round(block, keys[i])
            block = n2 + n1
        block = block[32:] + block[0:32]
        new_blocks.append(block)

    return ''.join(new_blocks)

In [5]:
def GOST_CBC_ENCRYPT(cbc_vector, key, message):
    blocks = split_to_blocks(message, 64)
    keys = None
    sub_keys = get_8sub_keys(key)
    keys = sub_keys * 3 + list(reversed(sub_keys))
    
    new_blocks = []
    for i in range(0, len(blocks)):
        block = blocks[i]
        block = xor_bits(block, cbc_vector)
        for rid in range(0, 32):
            n2, n1 = GOST_round(block, keys[rid])
            block = n2 + n1
        block = block[32:] + block[0:32]
        cbc_vector = block 
        new_blocks.append(block)

    return ''.join(new_blocks)


def GOST_CBC_DECRYPT(init_cbc_vector, key, message):
    blocks = split_to_blocks(message, 64)
    sub_keys = get_8sub_keys(key)
    keys = sub_keys + list(reversed(sub_keys)) * 3
    
    new_blocks = []
    cbc_vector = None
    for i in range(len(blocks) - 1, -1, -1):
        block = blocks[i]
        for rid in range(0, 32):
            n2, n1 = GOST_round(block, keys[rid])
            block = n2 + n1
        block = block[32:] + block[0:32]
        
        if i == 0:
            cbc_vector = init_cbc_vector
        elif i > 0:
            cbc_vector = blocks[i - 1]
            
        block = xor_bits(block, cbc_vector)

        new_blocks.append(block)
    
    new_blocks = list(reversed(new_blocks))
    
    return ''.join(new_blocks)

In [6]:
gost_256_key = '256_crypto_key_for_gost_28147-89'
gost_256_key_bits = stobits(gost_256_key)
print(f'len: {len(gost_256_key_bits)}')
print(f'key: {gost_256_key_bits}')

len: 256
key: 0011001000110101001101100101111101100011011100100111100101110000011101000110111101011111011010110110010101111001010111110110011001101111011100100101111101100111011011110111001101110100010111110011001000111000001100010011010000110111001011010011100000111001


In [7]:
input_msg = 'text for encrypting with russian gost 28147-89'
input_msg_bits = sto64(stobits(input_msg))
print(f'len: {len(input_msg_bits)}')
print(f'msg: {input_msg_bits}')

len: 384
msg: 011101000110010101111000011101000010000001100110011011110111001000100000011001010110111001100011011100100111100101110000011101000110100101101110011001110010000001110111011010010111010001101000001000000111001001110101011100110111001101101001011000010110111000100000011001110110111101110011011101000010000000110010001110000011000100110100001101110010110100111000001110010000000000000000


### GOST_ECB

In [8]:
encrypted_ECB = GOST_ECB(gost_256_key_bits, input_msg_bits)
print(f'len: {len(encrypted_ECB)}')
print(f'enc: {encrypted_ECB}')

len: 384
enc: 111100100100100100000110111011000110111000000101010110000101001110101000010011100011000000000000101100000010111100101001100100100011001011001011011011100010110000111100100110110001001000100011000010001001001000000111010011010011111010010001100010000101110111101101101111001000001010001110011111011000000000110011110000001100111001111100011010001011001101101101010100011011101101111011


In [9]:
decrypted_ECB = GOST_ECB(gost_256_key_bits, encrypted_ECB, encrypt=False)
print(f'len: {len(decrypted_ECB)}')
print(f'enc: {decrypted_ECB}')

len: 384
enc: 011101000110010101111000011101000010000001100110011011110111001000100000011001010110111001100011011100100111100101110000011101000110100101101110011001110010000001110111011010010111010001101000001000000111001001110101011100110111001101101001011000010110111000100000011001110110111101110011011101000010000000110010001110000011000100110100001101110010110100111000001110010000000000000000


In [10]:
print('ECB')
print(f'is decrypted == message {decrypted_ECB == input_msg_bits}\n')
print(f'decrypted message: {bitstos(decrypted_ECB).decode("utf-8")}')

ECB
is decrypted == message True

decrypted message: text for encrypting with russian gost 28147-89  


### GOST_CBC

In [11]:
init_cbc_vector = 'crypto42'
init_cbc_vector_bits = stobits(init_cbc_vector)
print(f'len: {len(init_cbc_vector_bits)}')
print(f'iv : {init_cbc_vector_bits}')

len: 64
iv : 0110001101110010011110010111000001110100011011110011010000110010


In [12]:
encrypted_CBC = GOST_CBC_ENCRYPT(init_cbc_vector_bits, gost_256_key_bits, input_msg_bits)
print(f'len: {len(encrypted_CBC)}')
print(f'enc: {encrypted_CBC}')

len: 384
enc: 010111011000010011111001011001110010110111000111111010001111110010110100100110000100010011100001101000001001111011111010011000000010100000011000000001001001110011110010110010101111011010100011111010001000111101010101100000101011111110100110100001110100100011001100100111010010110010011001011100000001010011110000110101110111101011101110000001000000001010010000011110010010001000001010


In [13]:
decrypted_CBC = GOST_CBC_DECRYPT(init_cbc_vector_bits, gost_256_key_bits, encrypted_CBC)
print(f'len: {len(decrypted_CBC)}')
print(f'enc: {decrypted_CBC}')

len: 384
enc: 011101000110010101111000011101000010000001100110011011110111001000100000011001010110111001100011011100100111100101110000011101000110100101101110011001110010000001110111011010010111010001101000001000000111001001110101011100110111001101101001011000010110111000100000011001110110111101110011011101000010000000110010001110000011000100110100001101110010110100111000001110010000000000000000


In [14]:
print('CBC')
print(f'is decrypted == message {decrypted_CBC == input_msg_bits}\n')
print(f'decrypted message: {bitstos(decrypted_CBC).decode("utf-8")}')

CBC
is decrypted == message True

decrypted message: text for encrypting with russian gost 28147-89  
